In [2]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
import dawg
from nltk import ngrams
from heapq import nlargest
from operator import itemgetter

with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
#     ngram_dict = {rows[0]:rows[1] for rows in reader}
    ngram_data=[tuple([line[0], int(line[1])]) for line in reader if int(line[1])>2]
ngram_dict = dawg.IntCompletionDAWG(ngram_data)
# prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")

# Importing all the necessary dictionaries
suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader if int(line[2])>2]
sortedpopulardict = dawg.IntCompletionDAWG(data)


data = []
ngram_data = []

## Change this for other top10k, top100k of nothing
top100k = suffixes.iloc[range(100000),:]
suffixes = []
top100ktuple = [tuple(x) for x in top100k.values]
# top100ktuple = [tuple([line["0"], int(line["counts"])])]
top100kdict = dawg.IntCompletionDAWG(top100ktuple)
top100k = []

/root/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:

def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = [0,0,0,0,0,0]

    for n in range(1,7):
        
        sixgrams = ngrams(query_input.split(), n)
        for grams in sixgrams:
            try:
                ngram_scores[n-1] = ngram_scores[n-1] + ngram_dict.get(' '.join(grams),0)
            except:
                pass
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [[get_single_ngram_features(x, ngram_dict)] for x in inputs];
    return results


In [4]:
# Read in training data
removed_session_training = pd.read_csv('../data/removed_session_training.csv', header=None, index_col=[0])
removed_session_training[1] = removed_session_training[1].str.replace('[\W_]+', ' ')

In [52]:
## Process training
training_queries = removed_session_training.reset_index(drop=True)
candidate_prefixes = []
right_query = []

amount_of_suffix_splits = 5
for j in range(len(training_queries)):
    current_query = str(training_queries[1][j])
    split_query = current_query.split(" ")
    suffix = ' '.join(split_query[1:])
    prefix = split_query[0] + " "
    
    for i in np.unique(np.linspace(0, len(suffix), amount_of_suffix_splits).astype(int)):
        candidate_prefixes.append(prefix + suffix[:i])
        right_query.append(current_query)
    
print("done")
candidate_prefixes

done


['www ',
 'www bon',
 'www bonsai ',
 'www bonsai wbff',
 'www bonsai wbff org',
 'loislaw ',
 'loislaw c',
 'loislaw co',
 'loislaw com',
 'www ',
 'www tabi',
 'www tabiecum',
 'www tabiecumming',
 'www tabiecummings com',
 'idx ',
 'idx tec',
 'idx techso',
 'idx techsolsc',
 'idx techsolsc com',
 'www ',
 'www brid',
 'www bridlean',
 'www bridleandbit',
 'www bridleandbit com',
 'gall ',
 'gall s',
 'gall sto',
 'gall ston',
 'gall stones',
 'gallstones ',
 'http ',
 'http www flickr com p',
 'http www flickr com photos 88145967 n',
 'http www flickr com photos 88145967 n00 24368586 in p',
 'http www flickr com photos 88145967 n00 24368586 in pool 32148876 n00',
 'href ',
 'href a href alt a http ww',
 'href a href alt a http www flickr com photos 8',
 'href a href alt a http www flickr com photos 88145967 n00 24368586',
 'href a href alt a http www flickr com photos 88145967 n00 24368586 in pool 32148876 n00',
 'http ',
 'http www flickr com p',
 'http www flickr com photos 88145

In [53]:
# Run this to clear memory
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

In [54]:
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

time_taken_top100k = 0
split_time = 0 
appendtime = 0
ngramtime = 0
real_query_time = 0
total_time_begin = time.time()
checkpoint_time = time.time()
with open('../data/training_feature_data.csv','w') as file:        
    for candidate_prefix_i in range(len(candidate_prefixes)):
        if candidate_prefix_i % 10000 == 0:
            end_checkpoint = time.time() - checkpoint_time
            print("Time between checkpoints: " + str(end_checkpoint))
            print("Total time elapsed: " + str(time.time() - total_time_begin))
            print(str(candidate_prefix_i) + '/' + str(len(candidate_prefixes)))
            checkpoint_time = time.time()

        candidate_prefix = candidate_prefixes[candidate_prefix_i]
        original_query = right_query[candidate_prefix_i]

    #     Getting the endterm for every prefix
        split_time_start = time.time()
        splitted = re.split('[\W_]+',candidate_prefix)
        if candidate_prefix[len(candidate_prefix)-1] == ' ':
            endterm = splitted[len(splitted)-2] + " "
        else:
            endterm = splitted[len(splitted)-1]
        split_time = split_time + time.time() - split_time_start

        # Creating the synthetic candidates
        start = time.time()
        top10suffix = nlargest(10, top100kdict.items(prefix=candidate_prefix), key=itemgetter(1))
        out = [i[0] for i in top10suffix]    
        preprefix = candidate_prefix[:-len(endterm)]
        outcombined1 = [preprefix + s for s in out]
        time_taken_top100k = time_taken_top100k + time.time() - start

        current_candidates = outcombined1

        # Getting the top # real queries
        realquerystart = time.time()
        if not(candidate_prefix == "www " or candidate_prefix == "http "):
            top50queries = nlargest(25, sortedpopulardict.items(prefix=candidate_prefix), key=itemgetter(1))
            keys_pop = [i[0] for i in top50queries]
            current_candidates.extend(keys_pop)
        real_query_time = real_query_time + time.time() - realquerystart
        
        current_relevant_candidates = [s == original_query for s in current_candidates]
        
        # Appending
#         append_time_start = time.time()
#         all_candidates.extend(current_candidates)    
#         relevant_candidate.extend()
#         qids.extend(np.ones(len(current_candidates)) * candidate_prefix_i)
#         appendtime = appendtime + time.time() - append_time_start

#         # Creating features
#         ngram_time_start = time.time()
#         ngram_features.extend(get_ngram_features(current_candidates, ngram_dict))
#         ngramtime = ngramtime + time.time() - ngram_time_start

        feature_vector = get_ngram_features(current_candidates, ngram_dict)
        for i in range(len(feature_vector)):
            file.write(str(feature_vector[i][0]) + ";" + str(int(current_relevant_candidates[i])) + ";" + str(current_candidates[i]) + ";" + str(candidate_prefix_i))
            file.write('\n')
total_time = time.time() - total_time_begin

print("Total time: " + str(total_time))
print("Top100k suffix time: " + str(time_taken_top100k))
print("Total split time: " + str(split_time))
print("Total append time: " + str(appendtime))
print("Total ngram time: " + str(ngramtime))
print("Total real query time: " + str(real_query_time))


Time between checkpoints: 0.0006551742553710938
Total time elapsed: 0.0007598400115966797
0/12741070
Time between checkpoints: 2.069267749786377
Total time elapsed: 2.07012939453125
10000/12741070
Time between checkpoints: 1.6720519065856934
Total time elapsed: 3.7422924041748047
20000/12741070
Time between checkpoints: 2.0105538368225098
Total time elapsed: 5.752933025360107
30000/12741070
Time between checkpoints: 1.895354986190796
Total time elapsed: 7.6483988761901855
40000/12741070
Time between checkpoints: 1.683424949645996
Total time elapsed: 9.331916332244873
50000/12741070
Time between checkpoints: 1.8682575225830078
Total time elapsed: 11.200265169143677
60000/12741070
Time between checkpoints: 1.8934693336486816
Total time elapsed: 13.093825578689575
70000/12741070
Time between checkpoints: 1.7504401206970215
Total time elapsed: 14.84435772895813
80000/12741070
Time between checkpoints: 1.7896637916564941
Total time elapsed: 16.634110689163208
90000/12741070
Time between che

Time between checkpoints: 2.563777208328247
Total time elapsed: 163.18376684188843
830000/12741070
Time between checkpoints: 2.2562241554260254
Total time elapsed: 165.44008827209473
840000/12741070
Time between checkpoints: 1.9993832111358643
Total time elapsed: 167.43956398963928
850000/12741070
Time between checkpoints: 1.9664208889007568
Total time elapsed: 169.40607285499573
860000/12741070
Time between checkpoints: 2.1583096981048584
Total time elapsed: 171.56447410583496
870000/12741070
Time between checkpoints: 2.0466575622558594
Total time elapsed: 173.61122345924377
880000/12741070
Time between checkpoints: 2.02012300491333
Total time elapsed: 175.63144040107727
890000/12741070
Time between checkpoints: 2.253964424133301
Total time elapsed: 177.88551926612854
900000/12741070
Time between checkpoints: 2.106945037841797
Total time elapsed: 179.9925673007965
910000/12741070
Time between checkpoints: 2.036874294281006
Total time elapsed: 182.02968382835388
920000/12741070
Time be

Time between checkpoints: 2.039034128189087
Total time elapsed: 338.3538122177124
1650000/12741070
Time between checkpoints: 1.6441657543182373
Total time elapsed: 339.9980900287628
1660000/12741070
Time between checkpoints: 1.7706067562103271
Total time elapsed: 341.76879119873047
1670000/12741070
Time between checkpoints: 1.6829216480255127
Total time elapsed: 343.4519553184509
1680000/12741070
Time between checkpoints: 1.8098149299621582
Total time elapsed: 345.26188015937805
1690000/12741070
Time between checkpoints: 1.8574020862579346
Total time elapsed: 347.1193778514862
1700000/12741070
Time between checkpoints: 1.7028071880340576
Total time elapsed: 348.822438955307
1710000/12741070
Time between checkpoints: 2.1063852310180664
Total time elapsed: 350.92891669273376
1720000/12741070
Time between checkpoints: 1.996436357498169
Total time elapsed: 352.92544531822205
1730000/12741070
Time between checkpoints: 1.9067840576171875
Total time elapsed: 354.8324909210205
1740000/12741070

Time between checkpoints: 2.653233289718628
Total time elapsed: 527.1894729137421
2480000/12741070
Time between checkpoints: 2.6585497856140137
Total time elapsed: 529.8481223583221
2490000/12741070
Time between checkpoints: 2.5774457454681396
Total time elapsed: 532.4256579875946
2500000/12741070
Time between checkpoints: 2.592210292816162
Total time elapsed: 535.017961025238
2510000/12741070
Time between checkpoints: 2.712824821472168
Total time elapsed: 537.7310202121735
2520000/12741070
Time between checkpoints: 3.0366992950439453
Total time elapsed: 540.7678122520447
2530000/12741070
Time between checkpoints: 2.6925101280212402
Total time elapsed: 543.4604501724243
2540000/12741070
Time between checkpoints: 2.663451910018921
Total time elapsed: 546.1239967346191
2550000/12741070
Time between checkpoints: 1.7289197444915771
Total time elapsed: 547.8530118465424
2560000/12741070
Time between checkpoints: 1.844900369644165
Total time elapsed: 549.698159456253
2570000/12741070
Time be

Time between checkpoints: 2.583557605743408
Total time elapsed: 692.8702049255371
3310000/12741070
Time between checkpoints: 2.173112392425537
Total time elapsed: 695.0434119701385
3320000/12741070
Time between checkpoints: 2.1822078227996826
Total time elapsed: 697.2257146835327
3330000/12741070
Time between checkpoints: 1.9799127578735352
Total time elapsed: 699.2057189941406
3340000/12741070
Time between checkpoints: 2.064544200897217
Total time elapsed: 701.2706227302551
3350000/12741070
Time between checkpoints: 2.2140722274780273
Total time elapsed: 703.484815120697
3360000/12741070
Time between checkpoints: 2.018200159072876
Total time elapsed: 705.5031251907349
3370000/12741070
Time between checkpoints: 2.389997959136963
Total time elapsed: 707.8932311534882
3380000/12741070
Time between checkpoints: 2.46402645111084
Total time elapsed: 710.3576488494873
3390000/12741070
Time between checkpoints: 2.4265172481536865
Total time elapsed: 712.7842626571655
3400000/12741070
Time bet

...

Time between checkpoints: 1.7710981369018555
Total time elapsed: 1885.0617434978485
9070000/12741070
Time between checkpoints: 2.2564780712127686
Total time elapsed: 1887.318348646164
9080000/12741070
Time between checkpoints: 1.8027904033660889
Total time elapsed: 1889.1212339401245
9090000/12741070
Time between checkpoints: 2.0347163677215576
Total time elapsed: 1891.1560447216034
9100000/12741070
Time between checkpoints: 1.870140790939331
Total time elapsed: 1893.0264167785645
9110000/12741070
Time between checkpoints: 1.9994182586669922
Total time elapsed: 1895.02592086792
9120000/12741070
Time between checkpoints: 1.780810832977295
Total time elapsed: 1896.8068251609802
9130000/12741070
Time between checkpoints: 1.9336824417114258
Total time elapsed: 1898.7406010627747
9140000/12741070
Time between checkpoints: 2.3093600273132324
Total time elapsed: 1901.050214767456
9150000/12741070
Time between checkpoints: 1.5741066932678223
Total time elapsed: 1902.6246190071106
9160000/12741

Time between checkpoints: 2.707613229751587
Total time elapsed: 2054.36763715744
9890000/12741070
Time between checkpoints: 2.2804176807403564
Total time elapsed: 2056.6482932567596
9900000/12741070
Time between checkpoints: 2.197011947631836
Total time elapsed: 2058.8453965187073
9910000/12741070
Time between checkpoints: 2.2201669216156006
Total time elapsed: 2061.0657970905304
9920000/12741070
Time between checkpoints: 2.331749200820923
Total time elapsed: 2063.3978040218353
9930000/12741070
Time between checkpoints: 2.4132542610168457
Total time elapsed: 2065.8111493587494
9940000/12741070
Time between checkpoints: 2.674825668334961
Total time elapsed: 2068.4860620498657
9950000/12741070
Time between checkpoints: 3.1359453201293945
Total time elapsed: 2071.62211894989
9960000/12741070
Time between checkpoints: 2.4598193168640137
Total time elapsed: 2074.082058906555
9970000/12741070
Time between checkpoints: 2.736288547515869
Total time elapsed: 2076.81845498085
9980000/12741070
Ti

Time between checkpoints: 2.280158758163452
Total time elapsed: 2223.8457984924316
10710000/12741070
Time between checkpoints: 1.8112645149230957
Total time elapsed: 2225.6571531295776
10720000/12741070
Time between checkpoints: 1.9961912631988525
Total time elapsed: 2227.653578519821
10730000/12741070
Time between checkpoints: 1.9158141613006592
Total time elapsed: 2229.5694856643677
10740000/12741070
Time between checkpoints: 1.8890204429626465
Total time elapsed: 2231.4585916996
10750000/12741070
Time between checkpoints: 2.0221011638641357
Total time elapsed: 2233.4807896614075
10760000/12741070
Time between checkpoints: 1.879004716873169
Total time elapsed: 2235.3598866462708
10770000/12741070
Time between checkpoints: 1.9263958930969238
Total time elapsed: 2237.2865302562714
10780000/12741070
Time between checkpoints: 1.700042724609375
Total time elapsed: 2238.9866857528687
10790000/12741070
Time between checkpoints: 1.8536019325256348
Total time elapsed: 2240.840381860733
108000

Time between checkpoints: 1.782876968383789
Total time elapsed: 2399.542214870453
11530000/12741070
Time between checkpoints: 1.712266445159912
Total time elapsed: 2401.2545738220215
11540000/12741070
Time between checkpoints: 1.7559864521026611
Total time elapsed: 2403.0106580257416
11550000/12741070
Time between checkpoints: 1.796907901763916
Total time elapsed: 2404.8076572418213
11560000/12741070
Time between checkpoints: 1.9009919166564941
Total time elapsed: 2406.7089302539825
11570000/12741070
Time between checkpoints: 1.8041765689849854
Total time elapsed: 2408.5131974220276
11580000/12741070
Time between checkpoints: 1.8228671550750732
Total time elapsed: 2410.336154937744
11590000/12741070
Time between checkpoints: 1.8922719955444336
Total time elapsed: 2412.228514432907
11600000/12741070
Time between checkpoints: 2.0414648056030273
Total time elapsed: 2414.2700793743134
11610000/12741070
Time between checkpoints: 2.0764505863189697
Total time elapsed: 2416.346628665924
11620

Time between checkpoints: 1.9487552642822266
Total time elapsed: 2552.265029668808
12340000/12741070
Time between checkpoints: 2.488678216934204
Total time elapsed: 2554.753799676895
12350000/12741070
Time between checkpoints: 1.907627820968628
Total time elapsed: 2556.6616559028625
12360000/12741070
Time between checkpoints: 2.1155903339385986
Total time elapsed: 2558.7773389816284
12370000/12741070
Time between checkpoints: 1.8090174198150635
Total time elapsed: 2560.586446285248
12380000/12741070
Time between checkpoints: 2.0938141345977783
Total time elapsed: 2562.6803526878357
12390000/12741070
Time between checkpoints: 2.150562286376953
Total time elapsed: 2564.831004858017
12400000/12741070
Time between checkpoints: 2.104501247406006
Total time elapsed: 2566.93559551239
12410000/12741070
Time between checkpoints: 2.0282204151153564
Total time elapsed: 2568.9639053344727
12420000/12741070
Time between checkpoints: 1.8973681926727295
Total time elapsed: 2570.861362218857
12430000/

In [40]:
feature_vector[i][0]

[5007075, 120466, 25682, 0, 0, 0]

In [ ]:
# query = input("Query: ")
# original_query = input("Original query: ")

# search_query = re.sub('[\W_]+',' ', query)
# original_query = re.sub('[\W_]+',' ', original_query)

# startt = time.time()
# splitted = re.split('[\W_]+',search_query)
# print(splitted)
# if query[len(query)-1] == ' ':
#     endterm = splitted[len(splitted)-2] + " "
# else:
#     endterm = splitted[len(splitted)-1]

# print("Endterm -> '" + endterm + "'")

# out = top100k[top100k['0'].str.startswith(endterm)].nlargest(10, 'counts')
# out2 = sortedpopular[sortedpopular['0'].str.startswith(search_query)].nlargest(40, 'counts')

# outcombined1 = query[:-len(endterm)] + out['0'].astype(str)
# time.time() - startt
# all_candidates = outcombined1.append(out2.iloc[:,0])
# relevant_candidate = (all_candidates == original_query).apply(float)

# train_input = pd.DataFrame({
#     'query': all_candidates,
#     'relevant': relevant_candidate,
#     'qid': 1
# })
  
# train_input['ngram_features'] = get_ngram_features(all_candidates, ngram_dict)
# print(train_input)


In [ ]:
metric = pyltr.metrics.err.ERR(highest_score=1, gain_type='identity')

model = pyltr.models.LambdaMART(
    metric=metric,
    verbose=1,
)

TX2 = ngram_features.reshape(-1, 6) 
Ty2 = np.array(relevant_candidate)
Tqids2 = np.array(qids)
model.fit(TX2, Ty2, Tqids2)

In [ ]:
with open('../data/train.txt') as trainfile, \
        open('../data/vali.txt') as valifile, \
        open('../data/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [ ]:
EX = TX2 
Ey = Ty2
Eqids = Tqids2

Epred = model.predict(EX)

In [ ]:
metric_MRR = MRR.MRR(highest_score=1, gain_type='identity', k=10)

print('Random ranking:' + str(metric_MRR.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric_MRR.calc_mean(Eqids, Ey, Epred)))

In [ ]:
import MRR

In [ ]:
sorted_pred_y = pyltr.util.sort.get_sorted_y(Ey, Epred)
evaluateMRR(Eqids, sorted_pred_y)

In [ ]:
import pyltr.metrics.err
pyltr.metrics.err.ERR.evaluate = evaluate(self, qid, targets):
    residual = 1.0
    result = 0.0
    for i, t in enumerate(targets[:self.k]):
        assert t <= self.highest_score
        sprob = self._get_satisfied_prob(t)
        result += residual * sprob / (1.0 + i)
        residual *= (1.0 - sprob)
        if residual < _EPS:
            break
    return result

In [ ]:
metric = pyltr.metrics.NDCG(gain_type='identity', k=10)
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))

inputs = ['hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye', 'hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye']
EX = get_ngram_features(inputs,ngram_dict)
EX = np.append([], EX)
EX = EX.reshape(-1, 6) 
Ey = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Eqids = [0,0,0,0,0,1,1,1,1,1]

Epred = model.predict(EX)
print(Epred)
print('Random ranking:' + str(metric.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))
output = pd.DataFrame({
    'inputs': inputs,
    'value': Epred
})
output.sort_values('value', ascending=False)

In [ ]:
Eqids